In [42]:
import pandas as pd
import sys
import csv
from statistics import median

In [43]:
batchsizes = [1,2,4,8,16]

In [44]:
data = pd.read_csv('ssr_final_results.csv')

In [45]:
data['project_reqd_builds'] = pd.to_numeric(data['project_reqd_builds'], downcast="float", errors='coerce')
data["project_missed_builds"] = pd.to_numeric(data["project_missed_builds"], downcast="float", errors='coerce')
data["project_saved_builds"] = pd.to_numeric(data["project_saved_builds"], downcast="float", errors='coerce')
data['batch_delays'] = pd.to_numeric(data["batch_delays"], downcast="float", errors='coerce')

In [46]:
delay_avg = []
delays = data['project_delays'].tolist()

for i in delays:
    if i == '[]':
        delay_avg.append(0)
    else:
        row = i[1:-1].split(', ')
        row = [int(x) for x in row]
        delay_avg.append(sum(row))
data['total_sbs_delay'] = delay_avg

In [47]:
data['total_batch_delay'] = data['batch_delays'].tolist()

In [48]:
project_list = set(data['project'].tolist())
batching_alg = set(data['algorithm'].tolist())

In [49]:
result_file = open('total_gfm_ssr_results.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'builds_reqd', 'batch_delays', 'sbs_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

84

In [51]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_sbs_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = reqd_list[i]
                    best[5] = batch_delay_list[i]
                    best[6] = delay_list[i]
                    best[7] = i+2
                    best_i = i
                    min_distance = distance

            res_writer.writerow(best)

In [54]:
result_file = open('ssr_sbs_delay_lists.csv', 'w')
res_headers = ['project', 'method', 'algorithm', 'batch_size', 'median_sbs_delays', 'median_batch_delays', 'confidence']
res_writer = csv.writer(result_file)
res_writer.writerow(res_headers)

86

In [55]:
lines = []

In [59]:
for p in project_list:
    p_name = p + '.csv'
    alg_scores = {}

    p_data = data[data['project'] == p]
    #print(p_data)
    for alg in batching_alg:

        alg_data = p_data[p_data['algorithm'] == alg]
        #print(alg_data)

        for b in batchsizes:

            if alg == 'BATCH4':
                if b != 4:
                    continue

            if alg == 'BATCHSTOP4':
                if b < 4:
                    continue

            batch_data = alg_data[ alg_data['batch_size'] == b]

            reqd_list = batch_data['project_reqd_builds'].tolist()
            delay_list = batch_data['total_sbs_delay'].tolist()
            batch_delay_list = batch_data['total_batch_delay'].tolist()
            ssr_delays = batch_data['project_delays'].tolist()
            min_distance = sys.maxsize

            best = [p_name, 'ssr', alg, b, 0, 0]
            best_i = 0

            for i in range(len(reqd_list)):
                distance = ((reqd_list[i]**2)+((delay_list[i]+batch_delay_list[i])**2))**0.5
                if distance < min_distance:
                    best[4] = ssr_delays[i]
                    #best[5] = median(ssr_delays[i])
                    best[5] = i+2
                    best_i = i
                    min_distance = distance

            lines.append(best)
            if alg == 'BATCH4':
                print(best)

['jruby.csv', 'ssr', 'BATCH4', 4, '[2, 7, 7, 4, 6, 3, 9, 4, 5, 8, 7, 6, 4, 3, 5, 7, 9, 8, 3, 9, 9]', 9]
['geoserver.csv', 'ssr', 'BATCH4', 4, '[3, 5, 2, 4, 2, 3, 5, 2, 5]', 5]
['heroku.csv', 'ssr', 'BATCH4', 4, '[13, 3, 5]', 19]
['cloudify.csv', 'ssr', 'BATCH4', 4, '[6, 16, 3, 5, 2, 12, 7, 9, 14, 5]', 18]
['graylog2-server.csv', 'ssr', 'BATCH4', 4, '[13, 4, 15, 19, 2, 4, 4]', 20]
['orbeon-forms.csv', 'ssr', 'BATCH4', 4, '[4, 5, 8, 9, 7, 4, 8, 2, 3, 6, 4, 6, 2]', 10]
['gradle.csv', 'ssr', 'BATCH4', 4, '[4, 2, 7, 11, 10, 7, 3, 6, 13, 5, 11, 10, 6, 4, 9]', 15]
['puppet.csv', 'ssr', 'BATCH4', 4, '[2, 4]', 16]
['concerto.csv', 'ssr', 'BATCH4', 4, '[10, 13, 9, 12, 9, 12, 3, 2, 7]', 14]
['rubinius.csv', 'ssr', 'BATCH4', 4, '[5, 5, 2, 6, 2, 6, 3, 5, 3, 3, 5, 4]', 8]
['sufia.csv', 'ssr', 'BATCH4', 4, '[6]', 19]
['cloud_controller_ng.csv', 'ssr', 'BATCH4', 4, '[4, 3, 2, 2, 4, 8, 8, 7, 2, 5, 5, 5, 7]', 8]
['loomio.csv', 'ssr', 'BATCH4', 4, '[4, 5, 7, 7, 9, 2, 8, 8, 5]', 10]
['vagrant.csv', 'ssr',

In [60]:
ssr_delay = pd.DataFrame(lines, columns=['project', 'method', 'algorithm', 'batch_size', 'sbs_delays', 'confidence'])

In [62]:
ssr_delay.to_csv('ssr_sbs_delay_list.csv')